In [2]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
from pyspark.sql.functions import input_file_name, lit, col, isnull
from pyspark.sql import functions as F

In [ ]:
#Create a spark Context class, with custom config
conf = SparkConf()
conf.set('spark.default.parallelism', 700)
conf.set('spark.sql.shuffle.partitions', 700)
conf.set('spark.driver.memory', '30g')
conf.set('spark.driver.cores', 8)
conf.set('spark.executor.cores', 8)
conf.set('spark.executor.memory', '30g')
sc = SparkContext.getOrCreate(conf)

23/07/28 22:22:22 WARN Utils: Your hostname, Hops-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.140 instead (on interface en0)
23/07/28 22:22:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/28 22:22:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
## Create spark session
spark = SparkSession.builder.master('local[*]').\
                config('spark.sql.debug.maxToStringFields', '100').\
                appName("ETFs Spark Airflow Docker").getOrCreate()

In [3]:

existing_schema = StructType([
    StructField("Date", StringType(), False),
    StructField("Open", FloatType(), False),
    StructField("High", FloatType(), False),
    StructField("Low", FloatType(), False),
    StructField("Close", FloatType(), False),
    StructField("Adj Close", FloatType(), False),
    StructField("Volume", FloatType(), False),
    StructField("Symbol", FloatType(), False),
    StructField("Security Name", FloatType(), False)

])

In [4]:
input_path = "../data/stocks_etfs/A.csv"
stock_df = spark.read.csv(input_path, header=True, schema=existing_schema)

In [5]:
stock_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Adj Close: float (nullable = true)
 |-- Volume: float (nullable = true)
 |-- Symbol: float (nullable = true)
 |-- Security Name: float (nullable = true)



In [78]:
meta_symbol = spark.read.csv("../data/symbols_valid_meta.csv", header=True)
symbol_mapping = meta_symbol.select("Symbol", "Security Name").rdd.collectAsMap()
symbol_name = os.path.splitext(os.path.basename(input_path))[0]


'A'

In [70]:
stock_df.show(10)

+----------+---------+---------+---------+---------+---------+---------+------+-------------+
|      Date|     Open|     High|      Low|    Close|Adj Close|   Volume|Symbol|Security Name|
+----------+---------+---------+---------+---------+---------+---------+------+-------------+
|1999-11-18|32.546494| 35.76538|28.612303|31.473534|27.068665|6.25463E7|  null|         null|
|1999-11-19| 30.71352|30.758226|28.478184|28.880543|24.838577|1.52341E7|  null|         null|
|1999-11-22|29.551144|31.473534| 28.65701|31.473534|27.068665|6577800.0|  null|         null|
|1999-11-23|30.400572|31.205294|28.612303|28.612303| 24.60788|5975600.0|  null|         null|
|1999-11-24|28.701717| 29.99821|28.612303|29.372318|25.261524|4843200.0|  null|         null|
|1999-11-26|29.238197|29.685265|29.148785|29.461731|25.338428|1729400.0|  null|         null|
|1999-11-29| 29.32761|30.355865|29.014664|30.132332|25.915169|4074700.0|  null|         null|
|1999-11-30| 30.04292| 30.71352|29.282904|30.177038|25.95361

23/07/26 01:25:58 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 7, schema size: 9
CSV file: file:///Users/hople/working_folder/Bootcamp_practices/ML_PIPELINE_AIRFLOW_SPARK_DOCKER/Dockerize_entire_workflow/dags/data/stocks_etfs/A.csv


In [71]:
stock_df = stock_df.withColumn("Symbol", F.lit(symbol_name))
stock_df = stock_df.withColumn("Security Name", F.lit(symbol_mapping.get(symbol_name)))

In [ ]:
stock_df.write.parquet

In [72]:
stock_df.show(10)

+----------+---------+---------+---------+---------+---------+---------+------+--------------------+
|      Date|     Open|     High|      Low|    Close|Adj Close|   Volume|Symbol|       Security Name|
+----------+---------+---------+---------+---------+---------+---------+------+--------------------+
|1999-11-18|32.546494| 35.76538|28.612303|31.473534|27.068665|6.25463E7|     A|Agilent Technolog...|
|1999-11-19| 30.71352|30.758226|28.478184|28.880543|24.838577|1.52341E7|     A|Agilent Technolog...|
|1999-11-22|29.551144|31.473534| 28.65701|31.473534|27.068665|6577800.0|     A|Agilent Technolog...|
|1999-11-23|30.400572|31.205294|28.612303|28.612303| 24.60788|5975600.0|     A|Agilent Technolog...|
|1999-11-24|28.701717| 29.99821|28.612303|29.372318|25.261524|4843200.0|     A|Agilent Technolog...|
|1999-11-26|29.238197|29.685265|29.148785|29.461731|25.338428|1729400.0|     A|Agilent Technolog...|
|1999-11-29| 29.32761|30.355865|29.014664|30.132332|25.915169|4074700.0|     A|Agilent Tech

In [73]:
stock_df.select("Security Name")

DataFrame[Security Name: string]

In [39]:
stock_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Adj Close: float (nullable = true)
 |-- Volume: float (nullable = true)
 |-- Symbol: string (nullable = false)
 |-- Security Name: string (nullable = false)



In [27]:
import pandas as pd
from save_parquet import save_parquet
import os

#retain features columns
features = ['Symbol', 'Security Name', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
#path to save processed dataset
path = '../data/processed_stocks_etfs/'
#read metal symbol files
metal_symbol = pd.read_csv('../data/symbols_valid_meta.csv')
metal_symbol = metal_symbol[['Symbol', 'Security Name']]
#correct some wrong spelling, coresponding to Stock file name
metal_symbol['Symbol'] = metal_symbol['Symbol'].str.replace('$', '-',regex=False)
metal_symbol['Symbol'] = metal_symbol['Symbol'].str.replace('.V', '',regex=False)
#creat mapping dictionary
symbol_mapping = metal_symbol.set_index('Symbol').to_dict()['Security Name']

def add_name(file):
    #print(symbol_mapping)
    name = os.path.splitext(os.path.basename(file))[0]
    df = pd.read_csv(file)
    df['Symbol'] = name
    sec_name = symbol_mapping[name]
    df['Security Name'] = sec_name
    #print(sec_name)
    #df.name = name
    #return df
    save_parquet(df[features], name, path)

In [33]:
from multiprocessing import cpu_count

stocks_path = '../data/stocks_etfs/'
#path = '../data/processed_stocks_etfs/'
from load_files import load_file
#list of loaded csv files will split into n_processor, for parralezation process
n_processor = cpu_count()
#get batches of data, list of list
preprocessing_list = load_file(n_processor, stocks_path, 'csv')


def data_processing():
    '''
    Takes batch number as input
    Map function add_name for every dataframe in batch number in preprocessing_list
    '''
    temp = list(map(add_name, preprocessing_list))

#data_processing()


In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
import os
from multiprocessing import cpu_count
from load_files import load_file #function load files into batches

#Create a spark Context class, with custom config to optimize the performance
#conf.set('spark.sql.adaptive.coalescePartitions.initialPartitionNum', 24)
#conf.set('spark.sql.adaptive.coalescePartitions.parallelismFirst', 'false')
#conf.set('spark.sql.files.minPartitionNum', 1)
conf = SparkConf()
conf.set('spark.default.parallelism', 700)
conf.set('spark.sql.shuffle.partitions', 700)
#conf.set('spark.sql.files.maxPartitionBytes', '500mb')
conf.set('spark.driver.memory', '30g')
conf.set('spark.driver.cores', 8)
conf.set('spark.executor.cores', 8)
conf.set('spark.executor.memory', '30g')
sc = SparkContext.getOrCreate(conf)

## Initialize SparkSession
spark = SparkSession.builder.master('local[*]').\
                config('spark.sql.debug.maxToStringFields', '100').\
                appName("ETFs Spark Airflow Docker").getOrCreate()


#stock dir
stocks_dir = "../data/stocks_etfs"
#processed data dir
processed_stocks_dir = "../data/processed_stocks_etfs"

#Mapping dict
meta_symbol = spark.read.csv("../data/symbols_valid_meta.csv", header=True)
symbol_mapping = meta_symbol.select("Symbol", "Security Name").rdd.collectAsMap()

#Define Schema for the data
existing_schema = StructType([
    StructField("Date", StringType(), False),
    StructField("Open", FloatType(), False),
    StructField("High", FloatType(), False),
    StructField("Low", FloatType(), False),
    StructField("Close", FloatType(), False),
    StructField("Adj Close", FloatType(), False),
    StructField("Volume", FloatType(), False),
    StructField("Symbol", FloatType(), False),
    StructField("Security Name", FloatType(), False)

])

def add_sym_sec_name(input_file):
    """
    Function adds Symbol and Security Name to stock file
    """
    # Read data from CSV into the DataFrame using the existing schema
    stock_df = spark.read.csv(input_file, header=True, schema=existing_schema)

    # Get Symbol name from input file
    symbol_name = os.path.splitext(os.path.basename(input_file))[0]

    # Adding Symbol and Security Name
    stock_df = stock_df.withColumn("Symbol", F.lit(symbol_name))
    stock_df = stock_df.withColumn("Security Name", F.lit(symbol_mapping.get(symbol_name)))

    # Save the preprocessed data to a parquet file
    #output_file = os.path.join(processed_stocks_dir, f"{symbol_name}_preprocessed.parquet")
    output_file = f"{processed_stocks_dir}/{symbol_name}_preprocessed.parquet"
    stock_df.write.parquet(output_file, mode="overwrite")


def preprocessing_data():
    '''
    Takes batch number as input
    Map function add_sym_sec_name for every dataframe in batch number in preprocessing_list
    '''
    #list of loaded csv files will split into n_processor, for parralezation process in Airflow
    n_processor = cpu_count()
    #get batches of data
    preprocessing_list = load_file(n_processor, stocks_dir, 'csv')
    #temp = list(map(add_sym_sec_name, ('../data/stocks_etfs/A.csv')))
    #print(preprocessing_list)

preprocessing_data()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/28 01:06:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
str(preprocessing_list[0])

NameError: name 'preprocessing_list' is not defined

In [24]:
spark.read.csv(str(preprocessing_list[0]), header=True, schema=existing_schema).show(10)

+----------+----------+----------+----------+----------+----------+--------+------+-------------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|Symbol|Security Name|
+----------+----------+----------+----------+----------+----------+--------+------+-------------+
|1988-02-04|       0.0|0.61728394| 0.5555556| 0.5555556|0.42403868| 38700.0|  null|         null|
|1988-02-05|       0.0|0.61728394| 0.5555556| 0.5555556|0.42403868|606300.0|  null|         null|
|1988-02-08|       0.0|0.61728394| 0.5555556| 0.5555556|0.42403868| 19000.0|  null|         null|
|1988-02-09|       0.0|0.61728394| 0.5555556| 0.5555556|0.42403868| 23100.0|  null|         null|
|1988-02-10|       0.0|0.61728394| 0.5555556| 0.5555556|0.42403868| 10000.0|  null|         null|
|1988-02-11|       0.0|0.61728394| 0.5555556| 0.5555556|0.42403868| 10900.0|  null|         null|
|1988-02-12|       0.0|0.61728394| 0.5555556| 0.5555556|0.42403868|     0.0|  null|         null|
|1988-02-16|       0

23/07/28 00:37:48 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 7, schema size: 9
CSV file: file:///Users/hople/working_folder/ML_PIPELINE_AIRFLOW_SPARK_DOCKER/dags/data/stocks_etfs/IPAR.csv


In [27]:
from load_files import load_file
from multiprocessing import cpu_count
stocks_dir = '../data/stocks_etfs/'
n_processor = cpu_count()
#get batches of data
preprocessing_list = load_file(n_processor, '../data/processed_stocks_etfs/', 'parquet')

preprocessing_list[0][0].stem
#pd.read_parquet(preprocessing_list[0][0])

'AAL_preprocessed'

In [2]:
from sparksession import initilize_sparksession
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType
from pyspark.sql import functions as F
spark = initilize_sparksession()

23/08/01 19:12:26 WARN Utils: Your hostname, Hops-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.140 instead (on interface en0)
23/08/01 19:12:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/01 19:12:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
processed_stock = spark.read.parquet('../data/processed_stocks_etfs/A_preprocessed.parquet')

In [3]:
processed_stock.printSchema()

root
 |-- Symbol: string (nullable = true)
 |-- Security Name: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Adj Close: float (nullable = true)
 |-- Volume: float (nullable = true)



In [4]:
processed_stock.show(10)

+------+--------------------+----------+---------+---------+---------+---------+---------+---------+
|Symbol|       Security Name|      Date|     Open|     High|      Low|    Close|Adj Close|   Volume|
+------+--------------------+----------+---------+---------+---------+---------+---------+---------+
|     A|Agilent Technolog...|1999-11-18|32.546494| 35.76538|28.612303|31.473534|27.068665|6.25463E7|
|     A|Agilent Technolog...|1999-11-19| 30.71352|30.758226|28.478184|28.880543|24.838577|1.52341E7|
|     A|Agilent Technolog...|1999-11-22|29.551144|31.473534| 28.65701|31.473534|27.068665|6577800.0|
|     A|Agilent Technolog...|1999-11-23|30.400572|31.205294|28.612303|28.612303| 24.60788|5975600.0|
|     A|Agilent Technolog...|1999-11-24|28.701717| 29.99821|28.612303|29.372318|25.261524|4843200.0|
|     A|Agilent Technolog...|1999-11-26|29.238197|29.685265|29.148785|29.461731|25.338428|1729400.0|
|     A|Agilent Technolog...|1999-11-29| 29.32761|30.355865|29.014664|30.132332|25.915169|4

In [5]:
# Calculate volume moving average using Window function for the last 30 days, including the current row
w_date = Window.partitionBy(F.lit(0)).orderBy(F.col('Date')).rowsBetween(-29, 0)
processed_stock = processed_stock.withColumn('vol_moving_avg', F.mean('Volume').over(w_date))
processed_stock = processed_stock.withColumn('vol_moving_avg', F.round('vol_moving_avg', 0))
# Calculate the rolling median for the 'Volume' column over the last 30 days
processed_stock = processed_stock.withColumn('adj_close_rolling_med', F.expr('percentile(Volume, 0.5)').over(w_date))
processed_stock = processed_stock.withColumn('adj_close_rolling_med', F.round('adj_close_rolling_med', 0))
#drop the first 30 days
featured_stock = processed_stock.withColumn("counter", F.monotonically_increasing_id())
w_counter = Window.partitionBy(F.lit(0)).orderBy("counter")
featured_stock = featured_stock.withColumn("index", F.row_number().over(w_counter))
featured_stock = featured_stock.filter(F.col("index") >= 30)
featured_stock = featured_stock.drop("counter", "index")

In [6]:
processed_stock.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Symbol#0, Security Name#1, Date#2, Open#3, High#4, Low#5, Close#6, Adj Close#7, Volume#8, round(vol_moving_avg#57, 0) AS vol_moving_avg#68, round(adj_close_rolling_med#79, 0) AS adj_close_rolling_med#92]
   +- Window [avg(Volume#8) windowspecdefinition(0, Date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -29, currentrow$())) AS vol_moving_avg#57, percentile(Volume#8, 0.5, 1, 0, 0, false) windowspecdefinition(0, Date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -29, currentrow$())) AS adj_close_rolling_med#79], [0], [Date#2 ASC NULLS FIRST]
      +- Sort [0 ASC NULLS FIRST, Date#2 ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(0, 8), ENSURE_REQUIREMENTS, [plan_id=28]
            +- FileScan parquet [Symbol#0,Security Name#1,Date#2,Open#3,High#4,Low#5,Close#6,Adj Close#7,Volume#8] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/hople/working_fo

In [9]:
featured_stock.show(10)

+------+--------------------+----------+---------+---------+---------+---------+---------+---------+--------------+---------------------+
|Symbol|       Security Name|      Date|     Open|     High|      Low|    Close|Adj Close|   Volume|vol_moving_avg|adj_close_rolling_med|
+------+--------------------+----------+---------+---------+---------+---------+---------+---------+--------------+---------------------+
|     A|Agilent Technolog...|1999-12-31| 56.86695|  57.1799|54.542202|55.302216|47.562416|1931100.0|     5739950.0|            2937500.0|
|     A|Agilent Technolog...|2000-01-03| 56.33047|56.464592|48.193848|51.502148| 44.29417|4674300.0|     3810883.0|            2937500.0|
|     A|Agilent Technolog...|2000-01-04|48.730328| 49.26681|46.316166|47.567955| 40.91059|4765000.0|     3461913.0|            2937500.0|
|     A|Agilent Technolog...|2000-01-05|47.389126|47.567955|43.141987| 44.61731|38.372894|5758600.0|     3434607.0|            2937500.0|
|     A|Agilent Technolog...|2000-

In [8]:
featured_stock.limit(40).toPandas().iloc[29:40]

,Symbol,Security Name,Date,Open,High,Low,Close,Adj Close,Volume,vol_moving_avg,adj_close_rolling_med
29,A,"Agilent Technologies, Inc. Common Stock",2000-02-11,54.721031,54.944565,53.513950,53.916309,46.370472,751700.0,1951013.0,1694350.0
30,A,"Agilent Technologies, Inc. Common Stock",2000-02-14,53.826897,55.033978,53.513950,54.676323,47.024109,837600.0,1914563.0,1519500.0
31,A,"Agilent Technologies, Inc. Common Stock",2000-02-15,54.765736,59.370529,54.765736,58.565807,50.369244,2017500.0,1826003.0,1519500.0
32,A,"Agilent Technologies, Inc. Common Stock",2000-02-16,57.224606,58.565807,56.777538,57.939915,49.830936,1801700.0,1727227.0,1519500.0
33,A,"Agilent Technologies, Inc. Common Stock",2000-02-17,57.939915,69.384834,55.525749,69.384834,59.674088,944900.0,1566770.0,1417000.0
34,A,"Agilent Technologies, Inc. Common Stock",2000-02-18,63.304722,68.580116,62.991776,67.060089,57.674717,4060000.0,1617623.0,1417000.0
35,A,"Agilent Technologies, Inc. Common Stock",2000-02-22,68.669525,69.653076,64.377686,65.450645,56.290516,2227900.0,1597900.0,1417000.0
36,A,"Agilent Technologies, Inc. Common Stock",2000-02-23,65.540054,71.888412,65.540054,70.815453,60.904503,1690000.0,1582620.0,1417000.0
37,A,"Agilent Technologies, Inc. Common Stock",2000-02-24,72.514305,82.573318,71.888412,76.359085,65.672264,2956300.0,1619300.0,1417000.0
38,A,"Agilent Technologies, Inc. Common Stock",2000-02-25,73.944923,81.500359,73.855507,77.342636,66.518181,2048200.0,1639913.0,1506700.0


In [55]:
import pandas as pd
pd.read_parquet('../data/featuresAdded_stocks_etfs/A.parquet').head(10)

,Symbol,Security Name,Date,Open,High,Low,Close,Adj Close,Volume,vol_moving_avg,adj_close_rolling_med
29,A,"Agilent Technologies, Inc. Common Stock",1999-12-31,56.866951,57.179901,54.542202,55.302216,47.562416,1931100,5.739950e+06,2937500.0
30,A,"Agilent Technologies, Inc. Common Stock",2000-01-03,56.330471,56.464592,48.193848,51.502148,44.294170,4674300,3.810883e+06,2937500.0
31,A,"Agilent Technologies, Inc. Common Stock",2000-01-04,48.730328,49.266811,46.316166,47.567955,40.910591,4765000,3.461913e+06,2937500.0
32,A,"Agilent Technologies, Inc. Common Stock",2000-01-05,47.389126,47.567955,43.141987,44.617310,38.372894,5758600,3.434607e+06,2937500.0
33,A,"Agilent Technologies, Inc. Common Stock",2000-01-06,44.080830,44.349072,41.577251,42.918453,36.911816,2534400,3.319900e+06,2750800.0
34,A,"Agilent Technologies, Inc. Common Stock",2000-01-07,42.247852,47.165592,42.203148,46.494991,39.987797,2819600,3.252447e+06,2701750.0
35,A,"Agilent Technologies, Inc. Common Stock",2000-01-10,49.356224,49.803291,48.327969,49.311516,42.410137,2148400,3.266413e+06,2701750.0
36,A,"Agilent Technologies, Inc. Common Stock",2000-01-11,49.311516,49.311516,47.523247,48.640915,41.833397,1855900,3.192453e+06,2559150.0
37,A,"Agilent Technologies, Inc. Common Stock",2000-01-12,48.640915,48.640915,45.824390,47.657368,40.987488,1429800,3.096447e+06,2431650.0
38,A,"Agilent Technologies, Inc. Common Stock",2000-01-13,48.909157,49.937412,47.210300,48.372677,41.602703,1134300,3.035680e+06,2294150.0


In [ ]:
import unittest
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

def calculate_volume_moving_average_and_median(df):
    # Calculate volume moving average using Window function for the last 30 days, including the current row
    window_spec = Window.orderBy(F.col('Date')).rowsBetween(-29, 0)
    df = df.withColumn('vol_moving_avg', F.avg('Volume').over(window_spec))

    # Calculate the rolling median for the 'Volume' column over the last 30 days
    w_date = Window.orderBy(F.col('Date')).rowsBetween(-29, 0)
    df = df.withColumn('vol_rolling_median', F.expr('percentile_approx(Volume, 0.5) OVER w_date'))

    return df

class TestVolumeMovingAverageAndMedian(unittest.TestCase):

    def setUp(self):
        self.spark = SparkSession.builder.appName("TestVolumeMovingAverageAndMedian").getOrCreate()

    def tearDown(self):
        self.spark.stop()

    def test_calculate_volume_moving_average_and_median(self):
        # Create a test DataFrame with 'Date' and 'Volume' columns
        data = [
            ("2023-07-01", 100),
            ("2023-07-02", 150),
            ("2023-07-03", 200),
            # Add more test data here...
        ]
        schema = ["Date", "Volume"]
        df = self.spark.createDataFrame(data, schema)

        # Calculate the volume moving average and rolling median using the function being tested
        result_df = calculate_volume_moving_average_and_median(df)

        # Assert the correctness of the results
        # You can add more specific assertions based on your expected results.
        # For example, if you know the expected values for certain dates, you can use assert statements to check them.
        self.assertTrue("vol_moving_avg" in result_df.columns)
        self.assertTrue("vol_rolling_median" in result_df.columns)
        self.assertEqual(result_df.count(), df.count())  # Ensure no rows are lost during processing
        self.assertGreater(result_df.filter(F.col("vol_moving_avg").isNull()).count(), 0)  # Check for null values in the moving average column
        self.assertGreater(result_df.filter(F.col("vol_rolling_median").isNull()).count(), 0)  # Check for null values in the rolling median column

if __name__ == "__main__":
    unittest.main()


In [2]:
#processed data dir
processed_stocks_dir = "../data/processed_stocks_etfs/"
#featured data dir
featured_stocks_dir = '../data/featuresAdded_stocks_etfs/'

from multiprocessing import cpu_count
from load_files import load_file
#list of loaded csv files will split into n_processor batches, for parralezation data processing in Airflow
n_processor = cpu_count()
#get batches of data
preprocessing_list = load_file(n_processor, processed_stocks_dir, 'parquet')

In [8]:
preprocessing_list[0][0]

PosixPath('../data/processed_stocks_etfs/AAL_preprocessed.parquet')

In [3]:
spark.read.parquet('../data/featuresAdded_stocks_etfs/A.parquet').show(10)

+------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------+------------------+---------------------+-----------------+
|Symbol|       Security Name|               Date|              Open|              High|               Low|             Close|         Adj Close| Volume|    vol_moving_avg|adj_close_rolling_med|__index_level_0__|
+------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------+------------------+---------------------+-----------------+
|     A|Agilent Technolog...|1999-12-31 00:00:00| 56.86695098876953|57.179901123046875| 54.54220199584961|55.302215576171875| 47.56241607666016|1931100|         5739950.0|            2937500.0|               29|
|     A|Agilent Technolog...|2000-01-03 00:00:00| 56.33047103881836| 56.46459197998047|    48.19384765625| 51.50214767456055| 44.29417037963867|4674300|

In [4]:
spark.read.parquet('../data/featuresAdded_stocks_etfs/A_featured.parquet/').show(10)

+------+--------------------+----------+------------------+-----------------+------------------+------------------+------------------+-------+--------------+---------------------+
|Symbol|       Security Name|      Date|              Open|             High|               Low|             Close|         Adj Close| Volume|vol_moving_avg|adj_close_rolling_med|
+------+--------------------+----------+------------------+-----------------+------------------+------------------+------------------+-------+--------------+---------------------+
|     A|Agilent Technolog...|1999-12-31| 56.86695098876953|57.17990112304688| 54.54220199584961| 55.30221557617188| 47.56241607666016|1931100|     5739950.0|            2937500.0|
|     A|Agilent Technolog...|2000-01-03| 56.33047103881836|56.46459197998047|    48.19384765625| 51.50214767456055| 44.29417037963867|4674300|     3810883.0|            2937500.0|
|     A|Agilent Technolog...|2000-01-04| 48.73032760620117|49.26681137084961|46.316165924072266| 47.